In [1]:
import csv
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import math

tqdm.pandas()

In [2]:
all_nets = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/all_nets.json'

parsed_nondups_val1 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val1_fix.txt'
parsed_nondups_train1 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train1_fix.txt'

parsed_nondups_val1_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val1.csv'
parsed_nondups_train1_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train1.csv'


# Convert to csv

In [3]:
with open(parsed_nondups_val1, 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('parsed_nondups_val1.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

with open(parsed_nondups_train1, 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('parsed_nondups_train1.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

train_hw = pd.read_csv(parsed_nondups_train1_csv)
val_hw = pd.read_csv(parsed_nondups_val1_csv)

# Neural Networks Dictionnary

In [4]:
with open(all_nets, 'r') as f:
    nets_dict = json.load(f)


## Max block size

In [5]:
max_blocks = 0
for k,nn in nets_dict.items():
    if len(nn)>max_blocks:
        max_blocks=len(nn)

max_blocks = max_blocks +1 # add fc layer at the end


# General information 

nets_dict_keys :0000 to 00531 (532)

Train : 453 different NN

Val : 50 different NN

In [6]:
def convert_NN_name(x):
    x=str(x)
    while len(str(x))<5 :
        x = '0'+str(x)
    return x

train_hw['name']=train_hw['name'].apply(lambda x : convert_NN_name(x))
val_hw['name']=val_hw['name'].apply(lambda x : convert_NN_name(x))

In [7]:
train_hw.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,G,C,B,J,name,bw_power,core_power,total_power
0,118.0,2.0,256.0,179968.0,2048.0,3.636011e+06,18770712.0,19987780.0,0.000533,00000,79.951120,21.330414,101.281534
1,87.0,2.0,512.0,99968.0,2048.0,2.325259e+06,14362735.0,21849702.0,0.000570,00000,87.398808,22.816285,110.215093
2,124.0,2.0,512.0,99968.0,3072.0,2.533459e+06,12739050.0,21849702.0,0.000543,00000,87.398808,21.716202,109.115010
3,80.0,2.0,1024.0,99968.0,1024.0,2.327509e+06,12155422.0,21849702.0,0.000684,00000,87.398808,27.343500,114.742308
4,86.0,2.0,512.0,60000.0,1536.0,1.728057e+06,16407889.0,28402776.0,0.000604,00000,113.611104,24.174307,137.785411


# Add neural network to dataframe

In [8]:
train_hw["NN"]=train_hw['name'].map(nets_dict)
val_hw["NN"]=val_hw['name'].map(nets_dict)

In [9]:
train_hw.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,G,C,B,J,name,bw_power,core_power,total_power,NN
0,118.0,2.0,256.0,179968.0,2048.0,3.636011e+06,18770712.0,19987780.0,0.000533,00000,79.951120,21.330414,101.281534,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [..."
1,87.0,2.0,512.0,99968.0,2048.0,2.325259e+06,14362735.0,21849702.0,0.000570,00000,87.398808,22.816285,110.215093,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [..."
2,124.0,2.0,512.0,99968.0,3072.0,2.533459e+06,12739050.0,21849702.0,0.000543,00000,87.398808,21.716202,109.115010,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [..."
3,80.0,2.0,1024.0,99968.0,1024.0,2.327509e+06,12155422.0,21849702.0,0.000684,00000,87.398808,27.343500,114.742308,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [..."
4,86.0,2.0,512.0,60000.0,1536.0,1.728057e+06,16407889.0,28402776.0,0.000604,00000,113.611104,24.174307,137.785411,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [..."


# Treat NN

### Calculate flops

In [10]:
# FIXME DIDNT COUNT ACTIVATION

def conv_flops(hout,cin,cout,k,tensor_in,skip=0):
    flops = 2*k*k*cin*hout*hout*cout
    if skip ==1:
        flops += tensor_in
    return flops

def dw_flops(hout,c,k,tensor_in,skip=0, mult=1):
    flops = 2*k*k*c*hout*hout
    if skip == 1:
        flops +=tensor_in
    return flops

def fcc_flops(i, j):
    return (2*i-1)*j

def calculate_flop(convtype,hin,hout,cin,cout,k,exp,tensor_in, skip=0, mult=1):
    if convtype=='conv':
        return conv_flops(hout,cin,cout,k, tensor_in, skip)
    elif convtype=='dw':
        return dw_flops(hout,cin,k,tensor_in,skip)
    elif convtype=='inv':
        hout_pointwise = hin
        return conv_flops(hout=hout_pointwise, cin=cin,cout=cin*exp,k=1, tensor_in=tensor_in, skip=skip) + \
        dw_flops(hout=hout,c=cin*exp,k=k,tensor_in=0,skip=0) + \
        conv_flops(hout=hout,cin=cin*exp,cout=cout,k=1, tensor_in=0, skip=0)


 ### Calculate weights

In [11]:

def conv_weights(cin,cout,k):
    return (k*k*cin+1)*cout

def dw_weights(cin,k,mult=1):
    #dw = equi cin=1 and multiply by cout
    #return conv_flops(hout,1,cin,k)*mult
    return (k*k+1)*cin

def fcc_weight(i, j):
    return
    #return (2*i-1)*j

def calculate_weights(convtype,cin,cout,k,exp,mult=1):
    if convtype=='conv':
        return conv_weights(cin,cout,k)
    elif convtype=='dw':
        return dw_weights(cin,k)
    elif convtype=='inv':
        return conv_weights(cin, cin*exp, 1)+ dw_weights(cin*exp,k,mult) + conv_weights(cin*exp,cout,1)

# Treat the neural Network

In [12]:

nb_classes= 1000


# def no_relu(NN_frame):  
#     NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)

def treat_NN(NN_frame, values_to_keep):
    fc_block = {'exp':1, 'c_out':1000, 's':1, 'k':1, 'skip':0,'convtype':'conv'}
    NN_frame=NN_frame.append(fc_block, ignore_index=True)

    #no_relu
    NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)

    NN_frame['c_in'] = np.roll(NN_frame['c_out'], 1)
    NN_frame['c_in'][0]=3 # RGB channels

    NN_frame['k2']=NN_frame['k'].apply(lambda x : x*x)
    NN_frame['hidden_dim']=NN_frame['exp']*NN_frame['c_in']

    NN_frame['h_in'] = np.nan
    NN_frame['h_in'][0]=224 # Size imagenet - hardcoded
    for i in range(1, len(NN_frame)):
        if NN_frame.loc[i-1, 's']==1:
            NN_frame.loc[i, 'h_in']=NN_frame.loc[i-1, 'h_in']
        elif NN_frame.loc[i-1, 's']==2:
            NN_frame.loc[i, 'h_in']=NN_frame.loc[i-1, 'h_in']/2
        else : 
            raise NameError('Dont know the paddind')


    NN_frame['h_out'] = np.roll(NN_frame['h_in'], -1)
    NN_frame['h_out'][NN_frame.last_valid_index()]=NN_frame['h_in'][NN_frame.last_valid_index()]/NN_frame.loc[NN_frame.last_valid_index(), 's']

    NN_frame['tensor_in']=NN_frame['c_in']*NN_frame['h_in']*NN_frame['h_in']
    NN_frame['tensor_out']=NN_frame['c_out']*NN_frame['h_out']*NN_frame['h_out']

    NN_frame['weights']=NN_frame.apply(lambda x : calculate_weights(x.convtype,x.c_in,x.c_out,x.k,x.exp), axis=1)
    NN_frame['FLOPS']= NN_frame.apply(lambda x : calculate_flop(x.convtype,x.h_in,x.h_out,x.c_in,x.c_out,x.k,x.exp, x.tensor_in,x.skip), axis=1)
    #return  NN_frame
    return  NN_frame.loc[:, values_to_keep]




## Example NN_treatment

In [13]:
conv_types = ['conv', 'dw', 'inv']
columns_names = ['exp','c_out','s','k','skip']
values_to_keep = ['FLOPS','weights','tensor_in','tensor_out','hidden_dim','k2', 'skip']

NN = nets_dict['00001']
NN_frame = pd.DataFrame(NN, columns = columns_names+ ['convtype',])
NN_00001 = treat_NN(NN_frame, values_to_keep)
NN_00001

,FLOPS,weights,tensor_in,tensor_out,hidden_dim,k2,skip
0,23708160.0,980,150528.0,439040.0,3,9,0
1,7902720.0,350,439040.0,439040.0,35,9,0
2,29854720.0,1224,439040.0,426496.0,35,1,0
3,181260800.0,11909,426496.0,28224.0,170,25,0
4,7620480.0,1314,28224.0,28224.0,45,9,0
5,3951360.0,1406,28224.0,8624.0,45,9,0
6,2673440.0,1826,8624.0,8624.0,55,9,0
7,2121504.0,3379,8624.0,2548.0,66,25,0
8,2293200.0,6019,2548.0,2548.0,78,49,0
9,2293200.0,6019,2548.0,2548.0,78,49,0


# Verify properperly calculate number of weights

## A) Initialize one NN : nets_dict[00001]

In [14]:
import tensorflow as tf
#import keras
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.utils.vis_utils import plot_model

from keras import backend as K
def relu6(x):
    return K.relu(x, max_value=6.0)

def _conv_block(inputs, filters, kernel, strides):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    return x

def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t
    cchannel = int(filters * alpha)
    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))
    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    if r:
        x = Add()([x, inputs])
    return x

def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    x=Activation(relu6)(inputs)
    x = _bottleneck(x, filters, kernel, t, alpha, strides)
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)
    x=Activation(relu6)(x)
    return x

def Net1(input_shape, k, alpha=1.0):
    inputs = Input(shape=input_shape)

    x= Conv2D(35, 3, padding='same', strides=(2,2))(inputs)
    x = DepthwiseConv2D((3,3),  activation='relu', padding='same')(x)
    x= Conv2D(34, 1, padding='same', strides=(1,1))(x)

    x = _inverted_residual_block(x, 9, (5, 5), t=5, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 9, (3, 3), t=5, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 11, (3, 3), t=5, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 11, (3, 3), t=5, alpha=alpha, strides=1, n=1)

    x = _inverted_residual_block(x, 13, (5, 5), t=6, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 13, (7, 7), t=6, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 13, (7, 7), t=6, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 13, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    x = _inverted_residual_block(x, 30, (3, 3), t=3, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 30, (3, 3), t=3, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 29, (3, 3), t=3, alpha=alpha, strides=1, n=1)
    last_filters = 1079

    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)

    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    #!!!!!!!!!
    x = Conv2D(k, (1, 1), padding='same')(x)
    #!!!!!!!!!
    output = Reshape((k,))(x)
    model = Model(inputs, output)
    # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model


if __name__ == '__main__':
    model_1 = Net1((224, 224, 3), 1000, 1.0)
    print(model_1.summary())

Using TensorFlow backend.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 35)      980       
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 112, 112, 35)      350       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 34)      1224      
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 34)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 170)     5950      
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (Non

# Model total param = Total params: 1,164,580

In [15]:

print(sum(NN_00001['weights']))


1164580


In [16]:
 
##### FIXME!!!!!!!!!!!
def get_flops():
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            model_2 = Net1((224, 224, 3), 1000, 1.0)
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
            # Optional: save printed results to file
            # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
            # opts['output'] = 'file:outfile={}'.format(flops_log_path)

            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

            return flops.total_float_ops
get_flops()

print(sum(NN_00001['FLOPS']))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
375115874.0


# NOT USED
## Distinguish columns between convtypes

In [17]:
def distinct_contypes(NN_frame, columns_names, conv_types):
    columns_names_augmented = [ f'{name}_{conv_type}' for conv_type in conv_types for name in columns_names]
    NN_frame_processed= pd.DataFrame(None, index=np.arange(len(NN)), columns=columns_names_augmented)
    ### Len(NN) or max_blocks???
    for conv_type in conv_types:
        columns_conv_type = [f'{columns_name}_{conv_type}' for columns_name in columns_names]
        NN_frame_processed[columns_conv_type] = NN_frame.loc[NN_frame['convtype']==conv_type, columns_names]
    return NN_frame_processed

# Get the final representation of the NN 

In [18]:

def no_relu(NN_frame):  
    NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)


def NN_to_dataframe(NN_list, columns_names, conv_types, values_to_keep):
    NN_frame = pd.DataFrame(NN_list, columns = columns_names+ ['convtype',])
    no_relu(NN_frame)
    return treat_NN(NN_frame, values_to_keep)

    #NN_distinct_contypes = distinct_contypes(NN_frame, columns_names, conv_types)
    #return NN_frame
    #return NN_distinct_contypes


NN_df = NN_to_dataframe(NN, columns_names, conv_types,values_to_keep)

## Add HW params to NN_dataframes

In [19]:
params = ['mac_num','mac_array_num','data_bits','sram_size','max_filter_size']

def add_HW_param_column(df,mac_num, mac_array_num, data_bits, sram_size, max_filter_size):
    df['mac_num']=mac_num
    df['mac_array_num']=mac_array_num
    df['total_macs']=  mac_num * mac_array_num
    df['data_bits']=data_bits
    df['sram_size']=sram_size
    df['max_filter_size']=max_filter_size


def add_HW_param_line(NN_frame, mac_num, mac_array_num, data_bits, sram_size, max_filter_size):
    HW_param_line =pd.DataFrame([[mac_num]*(NN_frame.shape[1])], columns=NN_frame.columns)
    HW_param_line = HW_param_line.append(pd.DataFrame([[mac_array_num]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line = HW_param_line.append(pd.DataFrame([[mac_num*mac_array_num]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[data_bits]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[sram_size]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[max_filter_size]*(NN_frame.shape[1])], columns=NN_frame.columns))
    return HW_param_line.append(NN_frame)


#not_used
def extend_to_max_blocks(NN_frame, value):
    for i in range (max_blocks-len(NN_frame)):
        NN_frame=NN_frame.append(pd.DataFrame([[0]*(NN_frame.shape[1])], index=[i+len(NN_frame),] ,columns=NN_frame.columns))
    return NN_frame

#not_used
def add_zero_blocks(arr):
    zero_blocks = np.zeros(( max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)



In [20]:
%pycache
train_hw= train_hw[:3]
val_hw = val_hw[:3]

UsageError: Line magic function `%pycache` not found.


# Apply and save

In [21]:
%pycache

values_to_keep = ['FLOPS','weights','tensor_in','tensor_out','hidden_dim','k2', 'skip']
name = '7'


# Convert NN to dataframe with values of interest
train_hw["NN_dataframe"] = train_hw["NN"].progress_apply(lambda x : NN_to_dataframe(x,columns_names, conv_types, values_to_keep))
#train_hw.progress_apply(lambda x : add_HW_param_column(x.NN_dataframe,x.mac_num,x.mac_array_num, x.data_bits, x.sram_size, x.max_filter_size), axis=1)
train_hw['NN_dataframe']=train_hw['NN_dataframe'].progress_apply(lambda x : np.array(x))
#complete to zero#complete to zero
#train_hw['NN_dataframe']= train['NN_dataframe'].progress_apply(lambda x : add_zero_blocks(x))#complete to zero#complete to zero
train_hw['NN_dataframe'].to_csv(f'train_{name}.csv')


#####################################

val_hw["NN_dataframe"] = val_hw["NN"].progress_apply(lambda x : NN_to_dataframe(x,columns_names, conv_types, values_to_keep))
#val_hw.progress_apply(lambda x : add_HW_param_column(x.NN_dataframe,x.mac_num,x.mac_array_num, x.data_bits, x.sram_size, x.max_filter_size), axis=1)
val_hw['NN_dataframe']=val_hw['NN_dataframe'].progress_apply(lambda x : np.array(x))

#complete to zero#complete to zero
#val['NN_dataframe']= val['NN_dataframe'].progress_apply(lambda x : add_zero_blocks(x))#complete to zero#complete to zero
val_hw.to_csv(f'val_{name}.csv')


UsageError: Line magic function `%pycache` not found.
